# Optiver's (possibly hungry) Ant

This is a brainteaser that was part of Optiver's recruiting process recently. The problem was the following:

## Problem
An ant leaves its anthill in order to forage for food. It moves with the speed of 10cm per second, but it doesn't know where to go, therefore every second it moves randomly 10cm directly north, south, east or west with equal probability.

a) If the food is located on east-west lines 20cm to the north and 20cm to the south, as well as on north-south lines 20cm to the east and 20cm to the west from the anthill, how long will it take the ant to reach it on average?

b) What is the average time the ant will reach food if it is located only on a diagonal line passing through (10cm, 0cm) and (0cm, 10cm) points?

c) Can you write a program that comes up with an estimate of average time to find food for any closed boundary around the anthill? What would be the answer if food is located outside an defined by $( (x – 2.5cm) / 30cm )^2 + ( (y – 2.5cm) / 40cm )^2 < 1$ in coordinate system where the anthill is located at $(x = 0cm, y = 0cm)$? Provide us with a solution rounded to the nearest integer.






---



## Solution
### a)
For problem a) we immediately notice that there are only three different positions the ant can pass by without hitting the border, however, it can travel between those three states arbitrarily long - with vanishing probability.

The three states x,y,z are: the center x, the orthogonally adjacent boxes y and finally the corners z. The watiting times will be identical for all boxes labeled y and all boxes labeled z due to symmetry reasons.

\begin{matrix} z & y & z\\ y & x & y \\ z & y & z \end{matrix}

Looking at the position x, what is the expectation of the waiting time $T_x$ at x to find food?
We do not know the exact number, but what we can say for sure is, the only possbile next state after moving north, east, south or west is state y. Therefore, whaterver the waiting time at state y is, the following equation holds:
\begin{align*}
 \mathbb{E}[T_x] = \mathbb{P}(x \rightarrow y)(\mathbb{E}[T_y] +1)
\end{align*}

In general for a state $a_0$ with adjacent states $a_1,...,a_4$, and $\mathbb{P}(a_0 \rightarrow a_i) = 0.25, i=1,...,4$ it holds
\begin{align*}
 \mathbb{E}[T_x] = \sum_{i=1}^4 \mathbb{P}(a_0 \rightarrow a_i)(\mathbb{E}[T_{a_i}] +1),
\end{align*}
where $\mathbb{E}[T_{a_i}] = 0$ if $a_i$ is part of the area where food is located.

Going back to the situation in a) from this rule we obtain three equations

(1) $\mathbb{E}[T_x] = \cdot(\mathbb{E}[T_y] +1)$

(2) $\mathbb{E}[T_y] = 0.25 (\mathbb{E}[T_x] +1) + 0.5 (\mathbb{E}[T_z] +1) + 0.25(0+1) $

(3) $\mathbb{E}[T_z] = 0.5 (\mathbb{E}[T_y] +1) + 0.5(0+1)$

This is a linear system of equations that can be solved for the three variables $\mathbb{E}[T_x], \mathbb{E}[T_y]$ and $\mathbb{E}[T_z]$ by solving
\begin{align*}
 \begin{pmatrix} 1 & -1 & 0 \\ -1/4 & 1 & -1/2 \\ 0 & -1/2 & 1 \end{pmatrix} \begin{pmatrix}\mathbb{E}[T_x]\\\mathbb{E}[T_y]\\\mathbb{E}[T_z] \end{pmatrix} = \begin{pmatrix}1\\1\\1 \end{pmatrix},
\end{align*}
which yields
\begin{align*}
  \begin{pmatrix}\mathbb{E}[T_x]\\ \mathbb{E}[T_y]\\ \mathbb{E}[T_z] \end{pmatrix} = \begin{pmatrix}4.5\\3.5\\2.75  \end{pmatrix}.
\end{align*}




---


### b)
The solution of exercise b) is quite interesting. We will use Doob's optional sampling theorem to proof the watining time is $\infty$.

Let's simplify notation and instead of 0cm, 10cm, 20cm,... we write 0,1,2,...
Then the food lies on the straight where x-coordinate + y-coordiante equals 1: $x+y=1$.

We start in position $(0,0)$, i.e., on the straight with $x+y=0$.
We will identify our position at step $t \in \mathbb{N}, Y_t$ by the value of $x+y$ of the straight we are currently on. This means $Y_0=0$. Further we realize that if we are in position $Y_t = z$, at the next step $t+1$  we arrive with a probability of 0.5 on straight $Y_{t+1} = z+1$ by going north or east, or on $Y_{t+1} = z-1$ by going south or west. Hence $Y_t$ is a random walk and more specifically a martingale.

The question of expected waiting time to arrive at the food corresponds to the epectation of waiting time $\tau = \inf \{ t| Y_t = 1\}$.

**Doob's optional stopping theorem says:**

For a martingal $Y_t$ with bounded increments (this is the case since $|Y_{t+1} - Y_t| = 1$) and a stopping time $\tau$ with finite expectation it holds, that:
\begin{align*}
  \mathbb{E}[Y_{\tau}] = \mathbb{E}[Y_0].
\end{align*}

This, however, is a problem since
\begin{align*}
  \mathbb{E}[Y_{\tau}] = Y_\tau =   1
\end{align*}
whereas
\begin{align*}
  \mathbb{E}[Y_0] = Y_0 = 0.
\end{align*}
This means one of the prerequisites of Doob's optional stopping theorem cannot be fulfilled. The only assumption that can fail is the fact that $\tau$ has finite expectation.

The only possible conclusion is:
\begin{align*}
  \mathbb{E}[\tau] = \infty.
\end{align*}






---


## c)

Now let's solve the case with an arbitrary closed boundary. We still write 0,1,2,... instead of 0cm,10cm,20cm,...

Assume that the boundary is handed to us as a matrix that contains ones where we are on or outside the food barrier, and zeros if we are inside.

For example the input we would get in problem a) looks like

\begin{matrix} 1 &1&1&1&1  \\ 1& 0 & 0 & 0 & 1\\ 1& 0 & 0 & 0 & 1 \\ 1& 0 & 0 & 0 &1 \\1 &1&1&1&1 \end{matrix}


Then the following code will calculate the exact solution of expected waiting times at all interior positions obtained by solving the system of equations obtained by the general relation between any interior state $a_{j_0}$ and its neighbor states $a_{j_i}, i=1,2,3,4$,
\begin{align*}
 \mathbb{E}[T_{a_{j_0}}] = \sum_{i=1}^4 \mathbb{P}(a_{j_0} \rightarrow a_{j_i})(\mathbb{E}[T_{a_{j_i}}] +1).
\end{align*}

*Remark: I directly calcualte the waiting times for all interior states. If only one state is of interest then the position indices i,j in the input matrix could be handed to the method as well and in the simulation it would make sense to start paths only in the state of interest. However, I did not bother to implement it, since my method includes this value anyway. Also I did not round to intergers, but only to two digit decimals, because I thought the values are quite interesting and rounding to integers hides a lot of information.*

In [ ]:
import numpy as np


def wait_matrix_analytic(grid_input):
    n_row, n_col = grid_input.shape
    dict_with_fields = {}
    field_counter = 0
    for i in range(n_row):
        for j in range(n_col):
            if grid_input[i, j] == 0:
                dict_with_fields[(i, j)] = field_counter
                dict_with_fields[field_counter] = (i, j)
                field_counter += 1

    A = np.zeros((field_counter, field_counter))

    for k in range(0, field_counter):
        (i, j) = dict_with_fields[k]
        A[k, k] = 1
        # check if i+-1,j+-1 is inside
        for neighbor in {(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)}:
            if grid_input[neighbor] == 0:
                col_id = dict_with_fields[neighbor]
                A[k, col_id] = - 0.25

    b = np.ones(field_counter)

    x = np.linalg.solve(A, b)
    waiting_matrix = grid_input * 0.0
    for k in range(0, field_counter):
        (i, j) = dict_with_fields[k]
        waiting_matrix[i, j] = x[k]

    return np.round(waiting_matrix, 2)


As a consistency check we also provide a method that calculates the expected waiting time at any state via a Monte-Carlo simulation. The additional parameter *n_sim_per_point* is the number of random walks that will be simulated starting from every interior state.

In [ ]:
def wait_matrix_simulation(grid_input, n_sim_per_point):
    waiting_matrix = grid_input * 0.0
    n_row, n_col = grid_input.shape
    dict_with_fields = {}
    field_counter = 0
    for i in range(n_row):
        for j in range(n_col):
            if grid_input[i, j] == 0:
                dict_with_fields[(i, j)] = field_counter
                dict_with_fields[field_counter] = (i, j)
                field_counter += 1

    for k in range(field_counter):
        steps = []
        for sim in range(n_sim_per_point):
            i, j = dict_with_fields[k]
            arrived = False
            step = 1
            while not arrived:
                direction = np.random.uniform()
                if direction <= 0.25:
                    i -= 1
                elif direction <= 0.5:
                    i += 1
                elif direction <= 0.75:
                    j -= 1
                else:
                    j += 1
                if grid_input[i, j] == 1:
                    arrived = True
                else:
                    step += 1
            steps.append(step)
        average_steps = np.array(steps).mean()
        waiting_matrix[dict_with_fields[k]] = average_steps
    return np.round(waiting_matrix, 2)



We compare the results for some specific input choices.

1. The example from exercise a)

In [ ]:
grid_input = np.array([[1, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 0, 0, 0, 1], [1, 0, 0, 0, 1],
                       [1, 1, 1, 1, 1]])
n_sim_per_point = 10000
print('Input:\n', grid_input)
print(f'Simulation with {n_sim_per_point} paths per position:\n', wait_matrix_simulation(grid_input=grid_input, n_sim_per_point=n_sim_per_point))
print('Analytic: \n', wait_matrix_analytic(grid_input=grid_input))

2. A variation of example a)

In [ ]:
grid_input = np.array([[1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1],
                       [1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1]])
n_sim_per_point = 10000
print('Input:\n', grid_input)
print(f'Simulation with {n_sim_per_point} paths per position:\n', wait_matrix_simulation(grid_input=grid_input, n_sim_per_point=n_sim_per_point))
print('Analytic: \n', wait_matrix_analytic(grid_input=grid_input))

3. A second asymmetric variation of example a)

In [ ]:
grid_input = np.array([[1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1], [1, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1],
                       [1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1]])
n_sim_per_point = 10000
print('Input:\n', grid_input)
print(f'Simulation with {n_sim_per_point} paths per position:\n', wait_matrix_simulation(grid_input=grid_input, n_sim_per_point=n_sim_per_point))
print('Analytic: \n', wait_matrix_analytic(grid_input=grid_input))

4. The grid proposed in exercise c) with $( (x – 0.25) / 3)^2 + ( (y – 0.25) / 4)^2 < 1$


In [ ]:
grid_input = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 0, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 1, 1],
    [1, 1, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1]])
n_sim_per_point = 10000
print('Input:\n', grid_input)
print(f'Simulation with {n_sim_per_point} paths per position:\n', wait_matrix_simulation(grid_input=grid_input, n_sim_per_point=n_sim_per_point))
print('Analytic: \n', wait_matrix_analytic(grid_input=grid_input))